# Videte Wheel
## Install Requirements

In [1]:
!conda install -y control slycot

Solving environment: done

# All requested packages already installed.



## Simple Pendulum

In [2]:
import sympy

# declare symbols
t = sympy.symbols('t')

x = sympy.Function('x')(t)
y = sympy.Function('y')(t)
θ = sympy.Function('θ')(t)

m, l, g = sympy.symbols('m l g')
θ_dot = θ.diff(t)

θ_dot

Derivative(θ(t), t)

In [3]:
L_tb = (m / 2) * l**2 * θ_dot**2 - m * g * l * sympy.cos(θ)
L = L_tb

L

-g*l*m*cos(θ(t)) + l**2*m*Derivative(θ(t), t)**2/2

In [4]:
f_θ = θ_dot

L_θ = sympy.diff(sympy.diff(L, sympy.diff(θ, t)), t) - sympy.diff(L, θ)
f_dθ = sympy.solve(L_θ, θ.diff(t, t))[0]

f_dθ

g*sin(θ(t))/l

In [5]:
states = [θ, θ.diff(t)]
eqs = [f_θ, f_dθ]

In [6]:
def createMatrix(eqs: list, states: list) -> sympy.Matrix:
    if (len(eqs) != len(states)):
        print("eqs and states must have the same size")
    A = sympy.zeros(len(eqs), len(eqs))
    for i, eq in enumerate(eqs, start=0):
        for j, state in enumerate(states, start=0):
            A[i, j] = sympy.diff(eq, state)
    return A

In [7]:
A = createMatrix(eqs, states)

A

Matrix([
[            0, 1],
[g*cos(θ(t))/l, 0]])

In [8]:
b = sympy.symbols('b') # b = {-1, 1}

linearice = [
    (sympy.sin(θ), 0),
    (sympy.cos(θ), b),
    (θ.diff(t, t), 0),
    (θ.diff(t), 0)
]

constants = [
    (m, 10),
    (g, 9.81),
    (l, 2)
]

In [9]:
import numpy as np

A_j_up = np.float64(sympy.simplify(A.subs(linearice).subs(constants).subs([(b, 1)])))
A_j_down = np.float64(sympy.simplify(A.subs(linearice).subs(constants).subs([(b, -1)])))

In [10]:
import control
import numpy.linalg as linalg

B = np.float64(sympy.Matrix([0, 1]))

C_up = control.ctrb(A_j_up, B)
C_down = control.ctrb(A_j_down, B)

rank_up = linalg.matrix_rank(C_up)
rank_down = linalg.matrix_rank(C_down)

print("Rank up:   " + str(rank_up))
print("Rank down: " + str(rank_down))

Rank up:   2
Rank down: 2


In [11]:
sympy.Matrix(control.obsv(A_j_up, C_up))

Matrix([
[  0.0, 1.0],
[  1.0, 0.0],
[4.905, 0.0],
[  0.0, 1.0]])

In [12]:
sympy.Matrix(control.obsv(A_j_down, C_down))

Matrix([
[   0.0, 1.0],
[   1.0, 0.0],
[-4.905, 0.0],
[   0.0, 1.0]])

In [13]:
Q = np.float64(np.diag([1, 1]))
R = np.float64(sympy.Matrix([0.1]))

sympy.Matrix(Q)

Matrix([
[1.0, 0.0],
[0.0, 1.0]])

In [14]:
print("A_down: " + str(A_j_down.shape) + ", A_up: " + str(A_j_up.shape) + ", B: " + str(B.shape) + ", Q: " + str(Q.shape) + ", R: " + str(R.shape))

A_down: (2, 2), A_up: (2, 2), B: (2, 1), Q: (2, 2), R: (1, 1)


## Generate LQR Controller

In [15]:
K, S, E = control.lqr(A_j_up, B, Q, R)

K_up = np.float64(K)

sympy.Matrix(K_up)

Matrix([[10.74101105208, 5.61088425332051]])

In [16]:
K, S, E = control.lqr(A_j_down, B, Q, R)

K_down = np.float64(K)

sympy.Matrix(K_down)

Matrix([[0.931011052080011, 3.44412864221998]])

## Simulation

In [17]:
import scipy.integrate as integrate
import math

dt = 0.01
timeline = np.arange(0., 1., dt)

def apply(y, t):
    return A.dot(y)

A = A_j_up
x_0 = np.float64([math.pi + 0.1, 0.0])

solution_up_lin = integrate.odeint(apply, x_0, timeline)

A = A_j_down
x_0 = np.float64([0.1, 0.0])

solution_down_lin = integrate.odeint(apply, x_0, timeline)

In [20]:
def applyA(y_k, t):
    print(str(y_k))
    print(t)
    print (A.shape)
    sys = np.float64(A).dot(y_k)
    KB = np.float64(K.T * B.T)
    print ("sys " + str(sys.shape))
    print (sys)
    print (B.shape)
    print (K.shape)
    print ("kb " + str(KB.shape))
    print (y.shape)
    contr = KB.dot(y_k)
    ret = sys - contr
    print ("contr " + str(contr.shape))
    print (contr)
    print (ret.shape)
    print (ret)
    return ret

A = A_j_up
K = K_up
x_0 = np.float64([math.pi + 0.1, 0.0])

solution_k_up_lin = integrate.odeint(applyA, x_0, timeline)

A = A_j_down
K = K_down
x_0 = np.float64([math.pi + 0.1, 0.0])

solution_k_down_lin = integrate.odeint(apply, x_0, timeline)

[3.24159265 0.        ]
0.0
(2, 2)
sys (2,)
[ 0.         15.90001197]
(2, 1)
(1, 2)
kb (2, 2)


AttributeError: 'y' object has no attribute 'shape'

In [ ]:
sys = A.dot(y)
KB = K * np.float64(B)

A

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(4)

for row, state in enumerate(states):
    axs[0].plot(timeline, solution_up_lin[:, row], label=str(state))

for row, state in enumerate(states):
    axs[2].plot(timeline, solution_down_lin[:, row], label=str(state))

for row, state in enumerate(states):
    axs[1].plot(timeline, solution_k_up_lin[:, row], label=str(state))

for row, state in enumerate(states):
    axs[3].plot(timeline, solution_k_down_lin[:, row], label=str(state))

In [ ]:
B